Using https://learnbybuilding.ai/tutorials/rag-from-scratch as a tutorial

In [1]:
""" Baby's First RAG """

corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]

def jaccard_similarity(query, document) -> float:
    """ Pre-processes plain strings into a set to perform comparisons.

    Args:
        query (_type_): _description_
        document (_type_): _description_

    Returns:
        float: _description_
    """
    query = query.lower().split(" ")
    document = document.lower().split(" ")
    intersection = set(query).intersection(set(document))
    union = set(query).union(set(document))
    return len(intersection)/len(union)

def return_response(query, corpus) -> str:
    """ Selects the best document to return to the user

    Args:
        query (_type_): _description_
        corpus (_type_): _description_

    Returns:
        str: _description_
    """
    similarities = []
    for doc in corpus:
        similarity = jaccard_similarity(user_input, doc)
        similarities.append(similarity)
    return corpus_of_documents[similarities.index(max(similarities))]



In [4]:
user_prompt = "What is a leisure activity that you like?"
user_input = "I like to be with friends"
return_response(user_input, corpus_of_documents)


'Have a picnic with friends and share some laughs.'

In [22]:
import requests
import json
import ollama
ollama.pull('mistral')


In [29]:

user_input = "I like to see my friends"
relevant_document = return_response(user_input, corpus_of_documents)
full_response = []
# https://github.com/jmorganca/ollama/blob/main/docs/api.md
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""

url = 'http://localhost:11434/api/generate'
data = {
    "model": "mistral:latest",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}


In [30]:
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)


In [31]:
try:
    count = 0
    for line in response.iter_lines():
        # filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            if 'response' in decoded_line:
                full_response.append(decoded_line['response'])
            else:
                print("Warning: 'response' key not found in the data:", decoded_line)
finally:
    response.close()
print(''.join(full_response))

 Plan a picnic with your friends for some quality time and shared laughter.


In [32]:
user_input = "I don't like to hike"
relevant_document = return_response(user_input, corpus_of_documents)
# https://github.com/jmorganca/ollama/blob/main/docs/api.md
full_response = []
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
Compile a recommendation to the user based on the recommended activity and the user input.
"""
url = 'http://localhost:11434/api/generate'
data = {
    "model": "mistral:latest",
    "prompt": prompt.format(user_input=user_input, relevant_document=relevant_document)
}
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            # print(decoded_line['response'])  # uncomment to results, token by token
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 If you don't like hiking, consider trying other outdoor activities such as having a picnic in a park or going for a leisurely walk instead.


This ends the tutorial at https://learnbybuilding.ai/tutorials/rag-from-scratch 
Now to go into https://learnbybuilding.ai/tutorials/rag-from-scratch-part-2-semantics-and-cosine-similarity

In [35]:
corpus_of_documents = [
    "Take a leisurely walk in the park and enjoy the fresh air.",
    "Visit a local museum and discover something new.",
    "Attend a live music concert and feel the rhythm.",
    "Go for a hike and admire the natural scenery.",
    "Have a picnic with friends and share some laughs.",
    "Explore a new cuisine by dining at an ethnic restaurant.",
    "Take a yoga class and stretch your body and mind.",
    "Join a local sports league and enjoy some friendly competition.",
    "Attend a workshop or lecture on a topic you're interested in.",
    "Visit an amusement park and ride the roller coasters."
]


{'status': 'success'}

In [37]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
doc_embeddings = model.encode(corpus_of_documents)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/home/kaminaduck/code/ai-holodeck/venv/lib/python3.10/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [38]:
print(doc_embeddings)

[[ 0.07121074 -0.01087999  0.11746483 ...  0.01414925 -0.13175759
  -0.00402594]
 [ 0.0488153  -0.03166638  0.07468719 ... -0.06278274 -0.11120293
   0.03045148]
 [ 0.05019969 -0.09127744  0.08517752 ...  0.01286448 -0.07415236
  -0.06140349]
 ...
 [ 0.05416269 -0.03030901  0.02475945 ... -0.01272298 -0.06512286
   0.05848261]
 [-0.00401893 -0.04562391 -0.00900756 ...  0.0393975  -0.12731639
   0.05255732]
 [ 0.05046043  0.01430451  0.08787955 ... -0.01778722 -0.0524641
  -0.02887328]]


In [43]:
from sklearn.metrics.pairwise import cosine_similarity
query = "What's the best activity to do with friends?"
query_embedding = model.encode([query])
similarities = cosine_similarity(query_embedding, doc_embeddings)
print(similarities)

[[0.40328795 0.36643532 0.3439691  0.3713997  0.6033779  0.27722937
  0.20104659 0.27300793 0.24499053 0.47120166]]


In [44]:
indexed = list(enumerate(similarities[0]))
sorted_index = sorted(indexed, key=lambda x: x[1], reverse=True)
print(sorted_index)

[(4, 0.6033779), (9, 0.47120166), (0, 0.40328795), (3, 0.3713997), (1, 0.36643532), (2, 0.3439691), (5, 0.27722937), (7, 0.27300793), (8, 0.24499053), (6, 0.20104659)]


In [45]:
recommended_documents = []
for value, score in sorted_index:
    formatted_score = "{:.2f}".format(score)
    print(f"{formatted_score} => {corpus_of_documents[value]}")
    if score > 0.3:
        recommended_documents.append(corpus_of_documents[value])

0.60 => Have a picnic with friends and share some laughs.
0.47 => Visit an amusement park and ride the roller coasters.
0.40 => Take a leisurely walk in the park and enjoy the fresh air.
0.37 => Go for a hike and admire the natural scenery.
0.37 => Visit a local museum and discover something new.
0.34 => Attend a live music concert and feel the rhythm.
0.28 => Explore a new cuisine by dining at an ethnic restaurant.
0.27 => Join a local sports league and enjoy some friendly competition.
0.24 => Attend a workshop or lecture on a topic you're interested in.
0.20 => Take a yoga class and stretch your body and mind.


In [48]:
prompt = """
You are a bot that makes recommendations for activities. You answer in very short sentences and do not include extra information.
These are potential activities:
{recommended_activities}
The user's query is: {user_input}
Provide the user with 2 recommended activities based on their query.
"""
recommended_activities = "\n".join(recommended_documents)
user_input = "I like to spend time with my friends"
full_prompt = prompt.format(user_input=user_input, recommended_activities=recommended_activities)
url = 'http://localhost:11434/api/generate'
data = {
    "model": "mistral:latest",
    "prompt": full_prompt
}
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
full_response=[]
try:
    count = 0
    for line in response.iter_lines():
        #filter out keep-alive new lines
        # count += 1
        # if count % 5== 0:
        #     print(decoded_line['response']) # print every fifth token
        if line:
            decoded_line = json.loads(line.decode('utf-8'))
            
            full_response.append(decoded_line['response'])
finally:
    response.close()
print(''.join(full_response))

 Have a picnic with friends.
Visit an amusement park together.


In [49]:
from unstructured.ingest.connector.local import SimpleLocalConfig
from unstructured.ingest.connector.weaviate import (
    SimpleWeaviateConfig,
    WeaviateAccessConfig,
    WeaviateWriteConfig,
)
from unstructured.ingest.interfaces import (
    ChunkingConfig,
    EmbeddingConfig,
    PartitionConfig,
    ProcessorConfig,
    ReadConfig,
)
from unstructured.ingest.runner import LocalRunner
from unstructured.ingest.runner.writers.base_writer import Writer
from unstructured.ingest.runner.writers.weaviate import (
    WeaviateWriter,
)


def get_writer() -> Writer:
    return WeaviateWriter(
        connector_config=SimpleWeaviateConfig(
            access_config=WeaviateAccessConfig(),
            host_url="http://localhost:8080",
            class_name="elements",
        ),
        write_config=WeaviateWriteConfig(),
    )


if __name__ == "__main__":
    writer = get_writer()
    runner = LocalRunner(
        processor_config=ProcessorConfig(
            verbose=True,
            output_dir="docs",
            num_processes=2,
        ),
        connector_config=SimpleLocalConfig(
            input_path="docs/Player_s Handbook.pdf",
        ),
        read_config=ReadConfig(),
        partition_config=PartitionConfig(),
        chunking_config=ChunkingConfig(chunk_elements=True),
        embedding_config=EmbeddingConfig(
            provider="langchain-huggingface",
        ),
        writer=writer,
        writer_kwargs={},
    )
    runner.run()

2024-04-16 19:50:21,058 MainProcess INFO     running pipeline: DocFactory -> Reader -> Partitioner -> Chunker -> Embedder -> Writer -> Copier with config: {"reprocess": false, "verbose": true, "work_dir": "/home/kaminaduck/.cache/unstructured/ingest/pipeline", "output_dir": "local-output-to-weaviate", "num_processes": 2, "raise_on_error": false}
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-04-16 19:50:21,071 MainProcess INFO     Running doc factory to generate ingest docs. Source connector: {"processor_config": {"reprocess": false, "verbose": true, "work_dir": "/home/kaminaduck/.cache/unstructured/ingest/pipeline", "output_dir": "local-output-to-weaviate", "num_processes": 2, "raise_on_error": false}, "read